<a href="https://colab.research.google.com/github/gaurav-adhikari/Hand_Gesture_Detector_TF/blob/master/Hands_Gesture_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [19]:
# Imports

import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

In [3]:
import os
for dirName, _ , signFiles in os.walk("/content/gdrive/My Drive/kaggle/SignLanguage"):
  for signFile in signFiles:
    print(os.path.join(dirName,signFile)) 

/content/gdrive/My Drive/kaggle/SignLanguage/amer_sign2.png
/content/gdrive/My Drive/kaggle/SignLanguage/amer_sign3.png
/content/gdrive/My Drive/kaggle/SignLanguage/american_sign_language.PNG
/content/gdrive/My Drive/kaggle/SignLanguage/sign_mnist_test.csv
/content/gdrive/My Drive/kaggle/SignLanguage/sign_mnist_train.csv
/content/gdrive/My Drive/kaggle/SignLanguage/sign_mnist_test/sign_mnist_test.csv
/content/gdrive/My Drive/kaggle/SignLanguage/sign_mnist_train/sign_mnist_train.csv


In [4]:
learning_rate=0.0001

In [5]:
train=pd.read_csv("/content/gdrive/My Drive/kaggle/SignLanguage/sign_mnist_train/sign_mnist_train.csv")
test=pd.read_csv("/content/gdrive/My Drive/kaggle/SignLanguage/sign_mnist_test/sign_mnist_test.csv")

X_train=train.drop("label",axis=1)
X_test=test.drop('label',axis=1)
y_train=train['label']
y_test=test['label']

X_train = (X_train/255).values
X_test = (X_test/255).values

In [6]:
encoder=OneHotEncoder()

y_train = encoder.fit_transform(y_train.values.reshape(-1,1)).toarray()
y_test = encoder.fit_transform(y_test.values.reshape(-1,1)).toarray()

In [7]:
((X_train.shape,y_train.shape),(X_test.shape,y_test.shape)) 

(((27455, 784), (27455, 24)), ((7172, 784), (7172, 24)))

In [8]:
def createCNNLayer(inputData,inputChannelsCount, filterCount, filterShape,poolShape, name):

  #Initialising filter
  convFilterShape=[filterShape[0],filterShape[1],inputChannelsCount,filterCount]

  weights= tf.Variable(tf.truncated_normal(convFilterShape,stddev=0.03),name=name+"_W")
  bias= tf.Variable(tf.truncated_normal([filterCount]),name=name+"_b")

  outLayer=tf.nn.conv2d(inputData,weights, [1,1,1,1],padding="SAME")

  outLayer+=bias

  outLayer = tf.nn.relu(outLayer)


  ksize = [1,poolShape[0],poolShape[1],1]

  stride=[1,2,2,1]

  outLayer=tf.nn.max_pool(outLayer,ksize,stride,padding="SAME")

  return outLayer

In [9]:
x=tf.placeholder(tf.float32,[None, 784])

X_Shaped=tf.reshape(x,[-1,28,28,1])

y=tf.placeholder(tf.float32,[None,10])

In [10]:
layer1= createCNNLayer(X_Shaped,1,32,[5,5],[2,2],"layer1")
layer2= createCNNLayer(layer1,32,64,[5,5],[2,2],"layer2")

flattenedArray=tf.reshape(layer2,[-1,7*7*64])

In [14]:
wd1 = tf.Variable(tf.truncated_normal([7*7*64,1000],stddev=0.03,name="wd1"))
bd1 = tf.Variable(tf.truncated_normal([1000],stddev=0.01,name="bd1"))
denseLayer1 = tf.nn.relu((tf.matmul(flattenedArray,wd1)+bd1))


wd2 = tf.Variable(tf.truncated_normal([1000,24],stddev=0.03,name="wd2"))
bd2 = tf.Variable(tf.truncated_normal([24],stddev=0.01,name="bd2"))
denseLayer2=tf.matmul(denseLayer1,wd2)+bd2
yhat = tf.nn.softmax(denseLayer2)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=denseLayer2,labels=y))

optimiser= tf.train.AdamOptimizer(learning_rate).minimize(loss)

corr_pred= tf.equal(tf.argmax(yhat, 1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(corr_pred,tf.float32))

In [18]:
with tf.Session() as sess:

  sess.run(tf.global_variables_initializer())

  numBatches = int(len(y_train)/50)

  for epoch in range(20):
    
    cost=0
    a=0

    for i in range(numBatches):
     
      batch_x = X_train[a:a+50]
      batch_y = y_train[a:a+50]

      _ , c= sess.run([optimiser,loss],feed_dict={x:batch_x,y:batch_y})

      cost += c/numBatches
      a+=50
    
    testDataAccuracy  = sess.run(accuracy,feed_dict={x:X_test,y:y_test})
    print("total cost at ", epoch+1," iteration ",cost," test Accuracy: ", testDataAccuracy)

print("Training Complete!!!")

total cost at  1  iteration  2.7009233538136073  test Accuracy:  0.44116005
total cost at  2  iteration  1.1873766756014297  test Accuracy:  0.6666202
total cost at  3  iteration  0.6279306844392534  test Accuracy:  0.7466537
total cost at  4  iteration  0.3677833430971165  test Accuracy:  0.7840212
total cost at  5  iteration  0.21965970895600906  test Accuracy:  0.817903
total cost at  6  iteration  0.1288323431514978  test Accuracy:  0.8400725
total cost at  7  iteration  0.07544509960993276  test Accuracy:  0.8637758
total cost at  8  iteration  0.0449125133878899  test Accuracy:  0.8781372
total cost at  9  iteration  0.02851189698477719  test Accuracy:  0.8839933
total cost at  10  iteration  0.019171554196154354  test Accuracy:  0.8898494
total cost at  11  iteration  0.010760671267869502  test Accuracy:  0.8912437
total cost at  12  iteration  0.008771823150214348  test Accuracy:  0.89626324
total cost at  13  iteration  0.010529990537954706  test Accuracy:  0.8929169
total cos